In [9]:
from sqlalchemy import Column, Integer, String, create_engine, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

In [10]:
from sqlalchemy.orm import sessionmaker

In [11]:
engine = create_engine('sqlite:///corpus.db', echo=True)

In [12]:
base = declarative_base()

In [13]:
class SejongCorpus(base):
    __tablename__ = 'SEJONG_CORPUS'
    
    pk = Column('pk', Integer, primary_key=True, autoincrement=True)
    ko_text = Column('ko_text', String, nullable=True)
    thai_text = Column('thai_text', String, nullable=True)
    file_name = Column('file_name', String, nullable=True)
    sent_count = Column('sent_count', Integer, nullable=True)
    cumsum_count = Column('cumsum_count', Integer, nullable=True)
    
    def __repr__(self):
        return 'pk:{}, ko_text:{}, thai_text:{}'.format(self.pk, self.ko_text, self.thai_text)

In [14]:
Session = sessionmaker(bind=engine)
session = Session()

In [15]:
ko_corpus = [r[0] for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all()]
thai_corpus = [r[1] for r in session.query(SejongCorpus.ko_text, SejongCorpus.thai_text).all()]

2020-08-06 23:51:15,588 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-06 23:51:15,591 INFO sqlalchemy.engine.base.Engine ()
2020-08-06 23:51:15,595 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-06 23:51:15,598 INFO sqlalchemy.engine.base.Engine ()
2020-08-06 23:51:15,600 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-06 23:51:15,622 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-06 23:51:15,641 INFO sqlalchemy.engine.base.Engine ()
2020-08-06 23:51:20,138 INFO sqlalchemy.engine.base.Engine SELECT "SEJONG_CORPUS".ko_text AS "SEJONG_CORPUS_ko_text", "SEJONG_CORPUS".thai_text AS "SEJONG_CORPUS_thai_text" 
FROM "SEJONG_CORPUS"
2020-08-06 23:51:20,139 INFO sqlalchemy.engine.base.Engine ()


In [16]:
len(ko_corpus), len(thai_corpus)

(500196, 500196)

In [17]:
with open('corpus.ko', 'w+', encoding='utf-8') as f:
    for ko in ko_corpus:
        f.write(ko+'|')

In [18]:
with open('corpus.th', 'w+', encoding='utf-8') as f2:
    for th in thai_corpus:
        f2.write(th+'|')